# Time Series Forecasting in Python:

## Objective:

* Combine, clean, and prepare the energy and temperature datasets for exploration and modeling. We will use the combined and cleaned datasets to make the exploration and modeling an easier task in the upcoming sections.

## Data Description:
* hr_temp_20170201-20200131_subset.csv – This is a dataset containing hourly (variable DATE) temperature data (variable HourlyDryBulbTemperature) at a weather station near the area you are forecasting energy for.

* hrl_load_metered - 20170201-20200131.csv – This is a dataset containing hourly (variable datetime_beginning_ept) megawatt usage data (variable mw) for the area in Pennsylvania centered around Duquesne. We are using only three years of data because we want to make sure that we look at recent energy patterns that are still applicable to our current customers.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/milestone1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/milestone1/hrl_load_metered - 20170201-20200131.csv
/kaggle/input/milestone1/hr_temp_20170201-20200131_subset.csv
/kaggle/input/milestone1/hr_temp_20200201-20200229_subset.csv


## 1.0 Load Data:
* First, let's make sure to import date related values with 'datetime64' data type for easy manipulation.

In [3]:
meter = pd.read_csv('/kaggle/input/milestone1/hrl_load_metered - 20170201-20200131.csv')#, parse_dates=['datetime_beginning_utc', 'datetime_beginning_ept'])
weather = pd.read_csv('/kaggle/input/milestone1/hr_temp_20170201-20200131_subset.csv', parse_dates=['DATE'])
weather1 = pd.read_csv('/kaggle/input/milestone1/hr_temp_20200201-20200229_subset.csv', parse_dates=['DATE'])

In [4]:
for column in ['datetime_beginning_utc', 'datetime_beginning_ept']:
    meter[column] = pd.to_datetime(meter[column])

weather = weather.sort_values('DATE')
meter = meter.sort_values('datetime_beginning_ept')
print(f'weather: {weather.shape}')
print(f'meter: {meter.shape}')

weather: (26280, 5)
meter: (26280, 8)


In [5]:
weather.dtypes

STATION                              int64
DATE                        datetime64[ns]
REPORT_TYPE                         object
SOURCE                               int64
HourlyDryBulbTemperature           float64
dtype: object

In [6]:
meter.dtypes

datetime_beginning_utc    datetime64[ns]
datetime_beginning_ept    datetime64[ns]
nerc_region                       object
mkt_region                        object
zone                              object
load_area                         object
mw                               float64
is_verified                         bool
dtype: object

## 2.0 Preprocessing:

### 2.1 Drop/Rename columns:
* Let's drop unnecessary columns and rename some columns for simplicity.

In [7]:
#weather
weather = weather.drop(columns=['STATION','REPORT_TYPE','SOURCE'])

In [8]:
#meter
meter = meter.rename(columns={'datetime_beginning_ept':'DATE'})
meter = meter.drop(columns=['datetime_beginning_utc','nerc_region','mkt_region','zone','load_area','is_verified'])
meter.head()

,DATE,mw
0,2017-02-01 00:00:00,1419.881
1,2017-02-01 01:00:00,1379.505
2,2017-02-01 02:00:00,1366.106
3,2017-02-01 03:00:00,1364.453
4,2017-02-01 04:00:00,1391.265


### 2.2 Index DateTime / Merge DataFrames:
* Let's create extract information (year, month, hour, day, day_of_week) from 'DATE'column.
* Then, we will create new columns for it.
* Fianlly, we will make these columns as index for each dataframe.
* This makes combining our two dataframes (energy & weather) easier using common index.

In [9]:
meter['day_of_week'] = meter.DATE.dt.dayofweek
meter['hour'] = meter.DATE.dt.hour
meter['day'] = meter.DATE.dt.day
meter['month'] = meter.DATE.dt.month
meter['year'] = meter.DATE.dt.year
meter.head(3)

,DATE,mw,day_of_week,hour,day,month,year
0,2017-02-01 00:00:00,1419.881,2,0,1,2,2017
1,2017-02-01 01:00:00,1379.505,2,1,1,2,2017
2,2017-02-01 02:00:00,1366.106,2,2,1,2,2017


In [10]:
weather['day_of_week'] = weather.DATE.dt.dayofweek
weather['hour'] = weather.DATE.dt.hour
weather['day'] = weather.DATE.dt.day
weather['month'] = weather.DATE.dt.month
weather['year'] = weather.DATE.dt.year
weather.head(3)

,DATE,HourlyDryBulbTemperature,day_of_week,hour,day,month,year
0,2017-02-01 00:53:00,37.0,2,0,1,2,2017
1,2017-02-01 01:53:00,37.0,2,1,1,2,2017
2,2017-02-01 02:53:00,36.0,2,2,1,2,2017


In [11]:
weather0 = weather.set_index(['year','month','day','hour','day_of_week'])
meter0 = meter.set_index(['year','month','day','hour','day_of_week'])

weather0 = weather0.drop(columns=['DATE'])
meter0 = meter0.drop(columns=['DATE'])
df = weather0.join(meter0, how='outer')
df = df.rename(columns={'HourlyDryBulbTemperature':'temp'})
df

temp        mw
year month day hour day_of_week                
2017 2     1   0    2            37.0  1419.881
               1    2            37.0  1379.505
               2    2            36.0  1366.106
               3    2            36.0  1364.453
               4    2            36.0  1391.265
...                               ...       ...
2020 1     31  19   4            34.0  1618.484
               20   4            33.0  1580.925
               21   4            33.0  1545.354
               22   4            33.0  1478.832
               23   4            34.0  1405.188

[26283 rows x 2 columns]

### 2.3 Identify Missing Data:
* Not all of the temperature data is recorded, as the stations would occasionally not report.
* So, we will fill in these missing values using linear interpolation.
* Let's look at what data we are mssing.

In [12]:
df.isna().sum()

temp    37
mw       3
dtype: int64

In [13]:
#missing mw values (Mar 12 2017, Mar 11 2018, Mar 10 2019)
# Sunday of 3rd week of March may be the yearly maintenance downtime for energy meter
missing_mw = df[df['mw'].isna()==True]
missing_mw

,,,,,temp,mw
year,month,day,hour,day_of_week,,
2017,3,12,2,6,17.0,NaN
2018,3,11,2,6,23.0,NaN
2019,3,10,2,6,46.0,NaN


In [14]:
# missing temp values (Oct 25 2018, Dec 25 2018, May 26 2019)
# weather temp sensor doesn't exibit any particular yearly maintenance downtime.
# we will fill this temp values with interpolation
missing_temp = df[df['temp'].isna()==True]
missing_temp

temp        mw
year month day hour day_of_week                
2018 10    25  12   3             NaN  1581.431
     12    25  2    1             NaN  1269.577
               3    1             NaN  1263.540
               4    1             NaN  1271.892
               5    1             NaN  1304.021
               6    1             NaN  1340.344
               7    1             NaN  1380.992
               8    1             NaN  1395.445
               9    1             NaN  1422.708
               10   1             NaN  1438.822
               11   1             NaN  1413.274
               12   1             NaN  1394.176
               13   1             NaN  1375.023
               14   1             NaN  1354.344
               15   1             NaN  1348.573
               16   1             NaN  1379.544
               17   1             NaN  1455.665
               18   1             NaN  1462.390
               19   1             NaN  1462.030
               20   1             NaN  1461.466
               21   1             NaN  1448.051
               22   1             NaN  1406.473
               23   1             NaN  1345.258
           26  0    2             NaN  1301.272
               1    2             NaN  1267.715
               2    2             NaN  1264.119
               3    2             NaN  1259.701
               4    2             NaN  1279.052
               5    2             NaN  1332.444
               8    2             NaN  1526.808
               9    2             NaN  1558.543
               10   2             NaN  1564.532
               11   2             NaN  1543.588
2019 5     26  1    6             NaN  1250.662
               2    6             NaN  1216.115
               3    6             NaN  1174.187
               7    6             NaN  1223.292

### 2.4 Treat Missing Data:
* we are missing:
    * 37 temperature data [temp (unit: degree F)]
    * 3 energy consumption data [mw (unit: MWh)]
* Let's use linear interpolation to fill in the gaps with forward direction.

In [15]:
df['temp'] = df['temp'].interpolate(method='linear', limit_direction = 'forward')
df['mw'] = df['mw'].interpolate(method='linear', limit_direction = 'forward')
df.isnull().sum()

temp    0
mw      0
dtype: int64

### 2.5 Re-index with Date:
* we will desolve multi-index (year, month, day, hour)
* Then, we will create a new index, 'date', which will combine all these values.

In [16]:
df = df.reset_index()
df.head(3)

,year,month,day,hour,day_of_week,temp,mw
0,2017,2,1,0,2,37.0,1419.881
1,2017,2,1,1,2,37.0,1379.505
2,2017,2,1,2,2,36.0,1366.106


In [17]:
df['date'] = pd.to_datetime(dict(year=df.year, month=df.month, day=df.day, hour=df.hour))
print(df.dtypes)
df.head(3)

year                    int64
month                   int64
day                     int64
hour                    int64
day_of_week             int64
temp                  float64
mw                    float64
date           datetime64[ns]
dtype: object


,year,month,day,hour,day_of_week,temp,mw,date
0,2017,2,1,0,2,37.0,1419.881,2017-02-01 00:00:00
1,2017,2,1,1,2,37.0,1379.505,2017-02-01 01:00:00
2,2017,2,1,2,2,36.0,1366.106,2017-02-01 02:00:00


In [18]:
df = df.set_index('date')
df = df.rename(columns={'day_of_week':'weekday'})
df.head(3)

,year,month,day,hour,weekday,temp,mw
date,,,,,,,
2017-02-01 00:00:00,2017,2,1,0,2,37.0,1419.881
2017-02-01 01:00:00,2017,2,1,1,2,37.0,1379.505
2017-02-01 02:00:00,2017,2,1,2,2,36.0,1366.106


### 2.6 Train/Test Split:
* train = from 2017-02-01 to 2019-12-31
* test = from 2020-01-01 to 2020-01-31

In [19]:
train = df[df.index.to_series().between('2017-01-01 00:00:00','2019-12-31 23:59:59') == True]
test = df[df.index.to_series().between('2020-01-01 00:00:00','2020-12-31 23:39:59') == True]

In [20]:
train

,year,month,day,hour,weekday,temp,mw
date,,,,,,,
2017-02-01 00:00:00,2017,2,1,0,2,37.0,1419.881
2017-02-01 01:00:00,2017,2,1,1,2,37.0,1379.505
2017-02-01 02:00:00,2017,2,1,2,2,36.0,1366.106
2017-02-01 03:00:00,2017,2,1,3,2,36.0,1364.453
2017-02-01 04:00:00,2017,2,1,4,2,36.0,1391.265
...,...,...,...,...,...,...,...
2019-12-31 19:00:00,2019,12,31,19,1,32.0,1606.134
2019-12-31 20:00:00,2019,12,31,20,1,32.0,1565.014
2019-12-31 21:00:00,2019,12,31,21,1,32.0,1515.905


In [21]:
test

,year,month,day,hour,weekday,temp,mw
date,,,,,,,
2020-01-01 00:00:00,2020,1,1,0,2,31.0,1363.428
2020-01-01 01:00:00,2020,1,1,1,2,29.0,1335.975
2020-01-01 02:00:00,2020,1,1,2,2,30.0,1296.817
2020-01-01 03:00:00,2020,1,1,3,2,30.0,1288.403
2020-01-01 04:00:00,2020,1,1,4,2,31.0,1292.263
...,...,...,...,...,...,...,...
2020-01-31 19:00:00,2020,1,31,19,4,34.0,1618.484
2020-01-31 20:00:00,2020,1,31,20,4,33.0,1580.925
2020-01-31 21:00:00,2020,1,31,21,4,33.0,1545.354


In [22]:
# let's confirm that we didn't miss any rows when splitting dataset to train/test
print(f'df: {df.shape[0]}')
print(f'train: {train.shape[0]}')
print(f'test: {test.shape[0]}')
print(f'train+test = df: {train.shape[0]+test.shape[0]} = {df.shape[0]}')

df: 26283
train: 25539
test: 744
train+test = df: 26283 = 26283


## Remark:
* Train (25539)/Test(744) split is successfully completed with no data loss.
* Now, we are ready to submit this assignment.

### 